In [ ]:
from pathlib import Path

In [ ]:
import json
import glob

In [ ]:
configs = []

for path in glob.glob(str(Path("output") / "config_*.json")):
    path = Path(path)

    with Path(path).open("r") as config:
        config = json.load(config)
    
    if config["output"] != "/dev/null":
        configs.append(config)

In [ ]:
import numpy as np

In [ ]:
def load_model_run(base_path: Path, ensemble_member: int, k: int) -> np.ndarray:
    file_path = base_path.with_name(f"{base_path.name}_{ensemble_member}")

    return np.fromfile(file_path, dtype=np.double).reshape(-1, k)

In [ ]:
def load_model_ensemble(base_path: Path, ensemble_size: int, k: int) -> np.ndarray:
    return np.stack([load_model_run(base_path, i, k=k) for i in range(ensemble_size)])

In [ ]:
def load_model_ensemble_from_config(config: dict) -> np.ndarray:
    return load_model_ensemble(Path(config["output"]), config["ensemble_size"], config["k"])

In [ ]:
def ensemble_mean(ensemble: np.ndarray) -> np.ndarray:
    return np.mean(ensemble, axis=0)

In [ ]:
def ensemble_spread(ensemble: np.ndarray) -> np.ndarray:
    return np.std(ensemble, axis=0)

In [ ]:
import matplotlib
import proplot as plt

In [ ]:
rates = sorted(set(c["zfp_fixed_rate"] for c in configs if c["compression_algorithm"] == "zfp"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble spread over time")

for rate, ax in zip(rates, axs):
    rate_configs = [
        c for c in configs
        if c["zfp_fixed_rate"] == rate
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "zfp"
    ]
    rate_configs = sorted(
        rate_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(rate_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in rate_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            np.mean(ensemble_spread(ensemble), axis=1),
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"ZFP(fixed_rate={config['zfp_fixed_rate']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("ensemble spread (mean over states 1..K)")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
bits = sorted(set(c["bitround_bits"] for c in configs if c["compression_algorithm"] == "bitround"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble spread over time")

for bit, ax in zip(bits, axs):
    bit_configs = [
        c for c in configs
        if c["bitround_bits"] == bit
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "bitround"
    ]
    bit_configs = sorted(
        bit_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(bit_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in bit_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            np.mean(ensemble_spread(ensemble), axis=1),
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"BitRound(keepbits={config['bitround_bits']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("ensemble spread (mean over states 1..K)")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
rates = sorted(set(c["zfp_fixed_rate"] for c in configs if c["compression_algorithm"] == "zfp"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble mean divergence (vs uncompressed) over time")

for rate, ax in zip(rates, axs):
    rate_configs = [
        c for c in configs
        if c["zfp_fixed_rate"] == rate
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "zfp"
    ]
    rate_configs = sorted(
        rate_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    base_ensemble = load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["zfp_fixed_rate"] == rate
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "zfp"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(rate_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in rate_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            (
                np.linalg.norm(ensemble_mean(ensemble) - base_ensemble_mean, axis=1)
                / np.sqrt(base_ensemble_mean.shape[-1])
            ),
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"ZFP(fixed_rate={config['zfp_fixed_rate']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("ensemble mean divergence")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
bits = sorted(set(c["bitround_bits"] for c in configs if c["compression_algorithm"] == "bitround"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble mean divergence (vs uncompressed) over time")

for bit, ax in zip(bits, axs):
    bit_configs = [
        c for c in configs
        if c["bitround_bits"] == bit
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "bitround"
    ]
    bit_configs = sorted(
        bit_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["bitround_bits"] == bit
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "bitround"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(bit_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in bit_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            (
                np.linalg.norm(ensemble_mean(ensemble) - base_ensemble_mean, axis=1)
                / np.sqrt(base_ensemble_mean.shape[-1])
            ),
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"BitRound(keepbits={config['bitround_bits']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("ensemble mean divergence")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
import CRPS.CRPS as pscore

rates = sorted(set(c["zfp_fixed_rate"] for c in configs if c["compression_algorithm"] == "zfp"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble Continuous Ranked Probability Score (vs uncompressed) over time")

for rate, ax in zip(rates, axs):
    rate_configs = [
        c for c in configs
        if c["zfp_fixed_rate"] == rate
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "zfp"
    ]
    rate_configs = sorted(
        rate_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["zfp_fixed_rate"] == rate
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "zfp"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(rate_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in rate_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"

        crps = []
        for i in range(ensemble.shape[1]):
            cs = []
            for k in range(ensemble.shape[2]):
                c = pscore(ensemble[:,i,k], base_ensemble_mean[i,k]).compute()[0]
                cs.append(c)
            crps.append(np.mean(cs))
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            crps,
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"ZFP(fixed_rate={config['zfp_fixed_rate']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("CRPS (mean over states 1..K)")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
import CRPS.CRPS as pscore

bits = sorted(set(c["bitround_bits"] for c in configs if c["compression_algorithm"] == "bitround"), reverse=True)

fig, axs = plt.subplots(nrows=1, ncols=len(rates))
fig.format(suptitle="Ensemble Continuous Ranked Probability Score (vs uncompressed) over time")

for bit, ax in zip(bits, axs):
    bit_configs = [
        c for c in configs
        if c["bitround_bits"] == bit
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "bitround"
    ]
    bit_configs = sorted(
        bit_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["bitround_bits"] == bit
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "bitround"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(bit_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for config in bit_configs:
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"

        crps = []
        for i in range(ensemble.shape[1]):
            cs = []
            for k in range(ensemble.shape[2]):
                c = pscore(ensemble[:,i,k], base_ensemble_mean[i,k]).compute()[0]
                cs.append(c)
            crps.append(np.mean(cs))
    
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            crps,
            label=label, cycle=cycle,
        )
    
    ax.set_title(f"BitRound(keepbits={config['bitround_bits']})")
    ax.set_xlabel("$t$")
    ax.set_ylabel("CRPS (mean over states 1..K)")
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[::-1], labels[::-1], title="compress ...", loc="right", ncols=1)
    
plt.show()

In [ ]:
rates = sorted(set(c["zfp_fixed_rate"] for c in configs if c["compression_algorithm"] == "zfp"), reverse=True)
frequencies = sorted(
    set(
        c["compression_frequency"] for c in configs
        if c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "zfp"
    ),
    key=lambda c: c if c == 0 else -1/c
)

fig, axs = plt.subplots(nrows=len(frequencies), ncols=len(rates))

axs.format(
    toplabels=[
        f"ZFP(fixed_rate={r})" for r in rates
    ],
    leftlabels=[
        "no compression" if c < 0
        else "compress only the output" if c == 0
        else f"compress state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
        for c in frequencies
    ],
    xlabel="$t$",
    ylabel="$X_{k=0}$",
)

axs = np.array(axs).reshape((len(frequencies), len(rates)))
fig.format(suptitle="Ensemble divergence (vs uncompressed) over time")

for rate, axc in zip(rates, axs.T):
    rate_configs = [
        c for c in configs
        if c["zfp_fixed_rate"] == rate
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "zfp"
    ]
    rate_configs = sorted(
        rate_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["zfp_fixed_rate"] == rate
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "zfp"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)
    base_ensemble_spread = ensemble_spread(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(rate_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for i, (config, ax) in enumerate(zip(rate_configs, axc)):
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"

        ax.fill_between(
            np.arange(ensemble.shape[-2]) * config["dt"],
            base_ensemble_mean[:,0] - base_ensemble_spread[:,0],
            base_ensemble_mean[:,0] + base_ensemble_spread[:,0],
            alpha=0.4, color="black",
        )
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            base_ensemble_mean[:,0],
            color="black", ls=":",
        )

        ax.fill_between(
            np.arange(ensemble.shape[-2]) * config["dt"],
            ensemble_mean(ensemble)[:,0] - ensemble_spread(ensemble)[:,0],
            ensemble_mean(ensemble)[:,0] + ensemble_spread(ensemble)[:,0],
            alpha=0.5, color=colors[i],
        )
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            ensemble_mean(ensemble)[:,0],
            label=label, color=colors[i],
        )
    
plt.show()

In [ ]:
bits = sorted(set(c["bitround_bits"] for c in configs if c["compression_algorithm"] == "bitround"), reverse=True)
frequencies = sorted(
    set(
        c["compression_frequency"] for c in configs
        if c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "bitround"
    ),
    key=lambda c: c if c == 0 else -1/c
)

fig, axs = plt.subplots(nrows=len(frequencies), ncols=len(bits))

axs.format(
    toplabels=[
        f"BitRound(keepbits={b})" for b in bits
    ],
    leftlabels=[
        "no compression" if c < 0
        else "compress only the output" if c == 0
        else f"compress state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"
        for c in frequencies
    ],
    xlabel="$t$",
    ylabel="$X_{k=0}$",
)

axs = np.array(axs).reshape((len(frequencies), len(bits)))
fig.format(suptitle="Ensemble divergence (vs uncompressed) over time")

for bit, axc in zip(bits, axs.T):
    bit_configs = [
        c for c in configs
        if c["bitround_bits"] == bit
        and c["compression_frequency"] >= 0
        and c["compression_algorithm"] == "bitround"
    ]
    bit_configs = sorted(
        bit_configs,
        key=lambda c: (
            c["compression_frequency"]
            if c["compression_frequency"] == 0
            else -1/c["compression_frequency"]
        ),
    )

    load_model_ensemble_from_config(
        next(
            c for c in configs
            if c["bitround_bits"] == bit
            and c["compression_frequency"] == -1
            and c["compression_algorithm"] == "bitround"
        )
    )
    base_ensemble_mean = ensemble_mean(base_ensemble)
    base_ensemble_spread = ensemble_spread(base_ensemble)

    colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, len(bit_configs)))
    cycle = plt.Cycle(colors=colors)
    
    for i, (config, ax) in enumerate(zip(bit_configs, axc)):
        ensemble = load_model_ensemble_from_config(config)

        c = config["compression_frequency"]
    
        if c < 0:
            label = "nothing"
        elif c == 0:
            label = "only the output"
        else:
            label = f"state every {c if c != 1 else 'step'}{' step' if c != 1 else ''}"

        ax.fill_between(
            np.arange(ensemble.shape[-2]) * config["dt"],
            base_ensemble_mean[:,0] - base_ensemble_spread[:,0],
            base_ensemble_mean[:,0] + base_ensemble_spread[:,0],
            alpha=0.4, color="black",
        )
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            base_ensemble_mean[:,0],
            color="black", ls=":",
        )

        ax.fill_between(
            np.arange(ensemble.shape[-2]) * config["dt"],
            ensemble_mean(ensemble)[:,0] - ensemble_spread(ensemble)[:,0],
            ensemble_mean(ensemble)[:,0] + ensemble_spread(ensemble)[:,0],
            alpha=0.5, color=colors[i],
        )
        ax.plot(
            np.arange(ensemble.shape[-2]) * config["dt"],
            ensemble_mean(ensemble)[:,0],
            label=label, color=colors[i],
        )
    
plt.show()

In [ ]:
perf_configs = []

for path in glob.glob(str(Path("output") / "config_*.json")):
    path = Path(path)

    with Path(path).open("r") as config:
        config = json.load(config)
    
    if config["performance"] != "/dev/null":
        perf_configs.append(config)

In [ ]:
import pandas as pd

In [ ]:
df = []

for config in perf_configs:
    algorithm = config["compression_algorithm"]
    device = "gpu" if config["compression_frequency"] > 0 else "cpu"
    points = config["k"] * config["ensemble_size"]
    zfp_fixed_rate = config["zfp_fixed_rate"] if algorithm == "zfp" else None
    bitround_bits = config["bitround_bits"] if algorithm == "bitround" else None
    
    skip = True
    
    with Path(config["performance"]).open("r") as file:
        for line in file.readlines():
            if skip:
                # skip the first entry to allow perf measuring warmup
                skip = not line.startswith("transfer_uncompressed")
                continue
            
            if line.startswith("compress"):
                df.append(dict(
                    algorithm=algorithm,
                    device=device,
                    points=points,
                    zfp_fixed_rate=zfp_fixed_rate,
                    bitround_bits=bitround_bits,
                    type="compress",
                    ms=float(line.split(" ")[2]),
                ))
            elif line.startswith("transfer_compressed"):
                df.append(dict(
                    algorithm=algorithm,
                    device=device,
                    points=points,
                    zfp_fixed_rate=zfp_fixed_rate,
                    bitround_bits=bitround_bits,
                    type="transfer_compressed",
                    ms=float(line.split(" ")[2]),
                ))
            elif line.startswith("decompress"):
                df.append(dict(
                    algorithm=algorithm,
                    device=device,
                    points=points,
                    zfp_fixed_rate=zfp_fixed_rate,
                    bitround_bits=bitround_bits,
                    type="decompress",
                    ms=float(line.split(" ")[2]),
                ))
            elif line.startswith("transfer_uncompressed"):
                df.append(dict(
                    algorithm=algorithm,
                    device=device,
                    points=points,
                    zfp_fixed_rate=zfp_fixed_rate,
                    bitround_bits=bitround_bits,
                    type="transfer_uncompressed",
                    ms=float(line.split(" ")[2]),
                ))
            else:
                print(line)
                assert False

df = pd.DataFrame(df)

In [ ]:
cpu_gpu_df = df.loc[
    ((df["points"] == 100000) | (df["points"] == 1000000) | (df["points"] == 10000000)) &
    ((df["zfp_fixed_rate"] == 16) | (df["bitround_bits"] == 16))
].groupby(["algorithm", "points", "type", "device"])["ms"]

pd.concat(
    [
        cpu_gpu_df.mean().rename("ms (mean)"),
        cpu_gpu_df.std().rename("ms (stdv)"),
    ],
    axis=1,
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

for typ, label in {
    "compress": "compress (GPU)",
    "transfer_compressed": "transfer GPU → CPU",
    "decompress": "decompress (GPU)",
}.items():
    df_typ = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "zfp") & (df["type"] == typ)
    ].groupby(["points"])[["points", "ms"]]
    
    ax.fill_between(
        df_typ.mean()["points"].values,
        (df_typ.mean()["ms"].values - df_typ.std()["ms"].values) / 1e3,
        (df_typ.mean()["ms"].values + df_typ.std()["ms"].values) / 1e3,
        alpha=0.15,
    )
    ax.plot(df_typ.mean()["points"].values, df_typ.mean()["ms"].values / 1e3, label=label)

ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("time [s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title("ZFP compression performance breakdown (averaged over ZFP's fixed rates)")
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

for typ, label in {
    "compress": "compress (GPU)",
    "transfer_compressed": "transfer GPU → CPU",
    "decompress": "decompress (GPU)",
}.items():
    df_typ = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "bitround") & (df["type"] == typ)
    ].groupby(["points"])[["points", "ms"]]
    
    ax.fill_between(
        df_typ.mean()["points"].values,
        (df_typ.mean()["ms"].values - df_typ.std()["ms"].values) / 1e3,
        (df_typ.mean()["ms"].values + df_typ.std()["ms"].values) / 1e3,
        alpha=0.15,
    )
    ax.plot(df_typ.mean()["points"].values, df_typ.mean()["ms"].values / 1e3, label=label)

ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("time [s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title("BitRound compression performance breakdown (averaged over BitRound's keepbits)")
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, 7))
cycle = plt.Cycle(colors=colors)

for zfp_fixed_rate in [8, 12, 16, 24, 32, 48, 64]:
    # sum all contributions to compression:
    #  compress, transfer_compressed, decompress
    df_rate = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "zfp") &
        (df["zfp_fixed_rate"] == zfp_fixed_rate) & (df["type"] != "transfer_uncompressed")
    ].reset_index(drop=True).rolling(window=3, step=1, closed="right")[["points", "ms"]].sum()[2::3]
    df_rate["points"] /= 3
    df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
    df_rate = df_rate.groupby(["points"])
    
    ax.fill_between(
        df_rate.mean().index.values,
        (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
        (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
        cycle=cycle, alpha=0.15,
    )
    ax.plot(
        df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
        cycle=cycle, label=str(zfp_fixed_rate),
    )

df_rate = df.loc[
    (df["device"] == "gpu") & (df["algorithm"] == "zfp") & (df["type"] == "transfer_uncompressed")
][["points", "ms"]].copy()
df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
df_rate = df_rate.groupby(["points"])

ax.fill_between(
    df_rate.mean().index.values,
    (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
    (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
    color="black", alpha=0.15,
)
ax.plot(
    df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
    color="black", ls=":", label="no compression",
)
    
ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("throughput [GB/s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title(
    "ZFP compression + transfer + decompression performance\n" +
    "vs no compression (transfer of uncompressed)"
)
ax.legend(title="ZFP fixed rate", ncols=1)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, 7))
cycle = plt.Cycle(colors=colors)

for bitround_bits in [8, 12, 16, 24, 32, 48, 52]:
    # sum all contributions to compression:
    #  compress, transfer_compressed, decompress
    df_rate = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "bitround") &
        (df["bitround_bits"] == bitround_bits) & (df["type"] != "transfer_uncompressed")
    ].reset_index(drop=True).rolling(window=3, step=1, closed="right")[["points", "ms"]].sum()[2::3]
    df_rate["points"] /= 3
    df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
    df_rate = df_rate.groupby(["points"])
    
    ax.fill_between(
        df_rate.mean().index.values,
        (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
        (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
        cycle=cycle, alpha=0.15,
    )
    ax.plot(
        df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
        cycle=cycle, label=str(bitround_bits),
    )

df_rate = df.loc[
    (df["device"] == "gpu") & (df["algorithm"] == "bitround") & (df["type"] == "transfer_uncompressed")
][["points", "ms"]].copy()
df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
df_rate = df_rate.groupby(["points"])

ax.fill_between(
    df_rate.mean().index.values,
    (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
    (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
    color="black", alpha=0.15,
)
ax.plot(
    df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
    color="black", ls=":", label="no compression",
)
    
ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("throughput [GB/s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title(
    "BitRound compression + transfer + decompression performance\n" +
    "vs no compression (transfer of uncompressed)"
)
ax.legend(title="BitRound keepbits", ncols=1)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, 7))
cycle = plt.Cycle(colors=colors)

for zfp_fixed_rate in [8, 12, 16, 24, 32, 48, 64]:
    # sum contributions to compression without decompression:
    #  compress, transfer_compressed
    df_rate = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "zfp") &
        (df["zfp_fixed_rate"] == zfp_fixed_rate) & (df["type"] != "transfer_uncompressed") &
        (df["type"] != "decompress")
    ].reset_index(drop=True).rolling(window=2, step=1, closed="right")[["points", "ms"]].sum()[1::2]
    df_rate["points"] /= 2
    df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
    df_rate = df_rate.groupby(["points"])
    
    ax.fill_between(
        df_rate.mean().index.values,
        (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
        (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
        cycle=cycle, alpha=0.15,
    )
    ax.plot(
        df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
        cycle=cycle, label=str(zfp_fixed_rate),
    )

df_rate = df.loc[
    (df["device"] == "gpu") & (df["algorithm"] == "zfp") & (df["type"] == "transfer_uncompressed")
][["points", "ms"]].copy()
df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
df_rate = df_rate.groupby(["points"])

ax.fill_between(
    df_rate.mean().index.values,
    (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
    (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
    color="black", alpha=0.15,
)
ax.plot(
    df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
    color="black", ls=":", label="no compression",
)
    
ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("throughput [GB/s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title(
    "ZFP compression + transfer (without decompression) performance\n" +
    "vs no compression (transfer of uncompressed)"
)
ax.legend(title="ZFP fixed rate", ncols=1)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

colors = matplotlib.pyplot.cm.coolwarm_r(np.linspace(0.0, 1.0, 7))
cycle = plt.Cycle(colors=colors)

for bitround_bits in [8, 12, 16, 24, 32, 48, 52]:
    # sum contributions to compression without decompression:
    #  compress, transfer_compressed
    df_rate = df.loc[
        (df["device"] == "gpu") & (df["algorithm"] == "bitround") &
        (df["bitround_bits"] == bitround_bits) & (df["type"] != "transfer_uncompressed") &
        (df["type"] != "decompress")
    ].reset_index(drop=True).rolling(window=2, step=1, closed="right")[["points", "ms"]].sum()[1::2]
    df_rate["points"] /= 2
    df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
    df_rate = df_rate.groupby(["points"])
    
    ax.fill_between(
        df_rate.mean().index.values,
        (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
        (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
        cycle=cycle, alpha=0.15,
    )
    ax.plot(
        df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
        cycle=cycle, label=str(bitround_bits),
    )

df_rate = df.loc[
    (df["device"] == "gpu") & (df["algorithm"] == "bitround") & (df["type"] == "transfer_uncompressed")
][["points", "ms"]].copy()
df_rate["throughput"] = (df_rate["points"] * 8) / (df_rate["ms"] / 1e3)
df_rate = df_rate.groupby(["points"])

ax.fill_between(
    df_rate.mean().index.values,
    (df_rate.mean()["throughput"].values - df_rate.std()["throughput"].values) / 1e9,
    (df_rate.mean()["throughput"].values + df_rate.std()["throughput"].values) / 1e9,
    color="black", alpha=0.15,
)
ax.plot(
    df_rate.mean().index.values, df_rate.mean()["throughput"].values / 1e9,
    color="black", ls=":", label="no compression",
)
    
ax.set_xlabel("k (number of points in the Lorenz96 model)")
ax.set_ylabel("throughput [GB/s]")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(plt.Formatter("log"))
ax.yaxis.set_major_formatter(plt.Formatter("log"))

ax.set_title(
    "BitRound compression + transfer (without decompression) performance\n" +
    "vs no compression (transfer of uncompressed)"
)
ax.legend(title="BitRound keepbits", ncols=1)

plt.show()